In [1]:
# import libraries
import pandas as pd
import numpy as np

In [2]:
# read csv file
df = pd.read_csv('./data/prep.csv')

# modeling
---------------------


## select modeling techuique
----------

今回のプロジェクトにおけるdata mining goalが**商品ページにその商品と高い確率で一緒に買われる製品をレコメンドする**ことであるため、バスケット分析を用いた製品レコメンドを行います。具体的にはmlxtendライブラリを用いてバスケット分析を実施していきます。

In [3]:
# pip install
#!pip install mlxtend

## generate test design
------------------

### オフライン評価

今回のプロジェクトではバスケット分析を実施するため、オフラインにおける主要な評価指標は`support`や`confidence`、`lift`になります。そのため、まずはオフライン評価でこれらの指標可視化して確認します。

### オンライン評価

現状、オンラインストアには商品ページに他の商品をレコメンドする場所はありません。そのため、ユーザに対してランダムに「レコメンドする」「レコメンドしない」と画面を出し分けるABテストを実施することで売上が何%向上するかを評価します。

- 注意点: 今回は実際のオンラインストアでオンライン評価を実施することが出来ないので、オンライン評価はスキップします。


## build model
----------


In [4]:
# import libraries
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [5]:
# parameters
min_support = 0.01
min_lift = 2

In [6]:
# apriori
freq_items = apriori(df, min_support=min_support, use_colnames=True)
freq_items.sort_values('support', ascending = False).reset_index(drop=True)

,support,itemsets
0,0.106334,(WHITE HANGING HEART T-LIGHT HOLDER)
1,0.091929,(REGENCY CAKESTAND 3 TIER)
2,0.086319,(JUMBO BAG RED RETROSPOT)
3,0.074450,(PARTY BUNTING)
4,0.074180,(ASSORTED COLOUR BIRD ORNAMENT)
...,...,...
968,0.010035,(SET 12 KIDS WHITE CHALK STICKS)
969,0.010035,(ABC TREASURE BOOK BOX )
970,0.010035,(3 DRAWER ANTIQUE WHITE WOOD CABINET)
971,0.010035,"(POPPY'S PLAYHOUSE LIVINGROOM , POPPY'S PLAYHO..."


### freq_items dataframeの説明

詳細は[こちらのウェブサイト](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/)を参照して下さい。

| column | 概要 |
|:---|:---|
| support | 日本語では**支持度**と言う。全トランザクション数のうちアイテム集合(itemsets)を含むトランザクション数の割合。アイテムAの支持度をsup(A)やsupport(A)と表される。 |
| itemsets | 日本語では**アイテム集合**と言う。アソシエーションルールでは、商品ではなくアイテムと言うことも多い。 |

In [7]:
# association rules
assoc_rules = association_rules(freq_items, metric="lift", min_threshold=min_lift)
assoc_rules.sort_values('lift', ascending=False).reset_index(drop=True)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(REGENCY TEA PLATE PINK),(REGENCY TEA PLATE GREEN ),0.012085,0.014566,0.010898,0.901786,61.909259,0.010722,10.033507
1,(REGENCY TEA PLATE GREEN ),(REGENCY TEA PLATE PINK),0.014566,0.012085,0.010898,0.748148,61.909259,0.010722,3.922605
2,(POPPY'S PLAYHOUSE LIVINGROOM ),"(POPPY'S PLAYHOUSE KITCHEN, POPPY'S PLAYHOUSE ...",0.013595,0.013703,0.010035,0.738095,53.863517,0.009848,3.765861
3,"(POPPY'S PLAYHOUSE KITCHEN, POPPY'S PLAYHOUSE ...",(POPPY'S PLAYHOUSE LIVINGROOM ),0.013703,0.013595,0.010035,0.732283,53.863517,0.009848,3.684512
4,(REGENCY SUGAR BOWL GREEN),(REGENCY MILK JUG PINK ),0.014458,0.014674,0.011114,0.768657,52.381694,0.010901,4.259150
...,...,...,...,...,...,...,...,...,...
907,(JUMBO BAG RED RETROSPOT),(LUNCH BAG SPACEBOY DESIGN ),0.086319,0.053302,0.011707,0.135625,2.544479,0.007106,1.095240
908,(REGENCY CAKESTAND 3 TIER),(SET OF 3 CAKE TINS PANTRY DESIGN ),0.091929,0.061826,0.011761,0.127934,2.069275,0.006077,1.075807
909,(SET OF 3 CAKE TINS PANTRY DESIGN ),(REGENCY CAKESTAND 3 TIER),0.061826,0.091929,0.011761,0.190227,2.069275,0.006077,1.121389
910,(WHITE HANGING HEART T-LIGHT HOLDER),(LUNCH BAG PINK POLKADOT),0.106334,0.050227,0.010682,0.100457,2.000069,0.005341,1.055839


### assoc_rules dataframeの説明

詳細は[こちらのウェブサイト](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/)を参照して下さい。

オムツを買う人はビールも買う({Diapers} -> {Beer})という関係性を例に説明します。

| column | 概要 |
|:---|:---|
| antecedents | 日本語では**条件部**と言う。{Diapers}はantecedentsとなる |
| consequents | 日本語では**結論部**と言う。{Beer}はconsequentsとなる |
| antecedents support | 条件部の支持度({Diapers}の支持度) |
| consequents support | 結論部の支持度({Beer}の支持度) |
| support | {Diapers} ∪ {Beer}の支持度 |
| confidence | 日本語では**確信度**と言う。この値が高いほど抽出したルールがオムツを買う人はビールも買うという関係の確信度はsup({Diapers} ∪ {Beer}) / sup({Diapers})で算出される。オムツを買う人がビールも買う確信度はconfidence({Diapers} -> {Beer})と表される。 |
| lift | 日本語では**リフト値**と言う。もしオムツとビールの購買関係が統計的に独立であった場合に比べて、どのくらいオムツを買う人はビールも買うという関係が多いのかを測る指標。リフト値はconfidence({Diapers} -> {Beer}) / support({Beer})で算出される |
| leverage | 詳しく知りたい方はウェブサイトを参照して下さい |
| conviction | 詳しく知りたい方はウェブサイトを参照して下さい |

## assess model
-------------

今回のプロジェクトではアソシエーションルールを用いて、支持度、確信度、リフト値を算出しました。一般的な分析プロジェクトでは、オンラインストアの担当者に算出した支持度、確信度、リフト値を報告し、算出結果が正しそうか、ルールの中に使えそうなルールは無いか、使えそうなルールがある場合どのようにオンラインストア上で表出すると良いかなどを議論します。

- 注意点: 今回はデモンストレーションのためスキップしますが、結果をきちんとレポートとしてまとめて協業者に報告するのもデータサイエンティストの仕事です。
- 注意点: 構築したモデルや分析結果は必ずドメインの専門家に見てもらうようにしましょう。ドメインの専門家に見てもらうことで、思わぬバグを見つけることが出来たり、モデルや分析結果に対する違和感、改善のためのアドバイスをもらうことが出来ます。

## note/questions
-------------

#### select modeling technique

#### generate test design

#### build model

#### assess model
